# Preparation

## Install

In [ ]:
!pip install matplotlib japanize-matplotlib

In [ ]:
!pip install seaborn

In [ ]:
! git clone https://github.com/google-deepmind/alphagenome.git
! pip install ./alphagenome

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

## .fa.gz -> .txt -> single_line.txt

In [ ]:
import os
import gzip

# 2番から22番染色体、およびX, Y染色体を処理するリスト
chromosomes = list(range(1, 23)) + ['X', 'Y']

for n in chromosomes:
    # 読み込むFASTAファイル名を.fa.gzに変更
    source_fa_file = f'references_GRCh37_chr{n}.fa.gz'
    destination_txt_file = f'references_GRCh37_chr{n}.txt'
    
    print(f"Processing {source_fa_file} -> {destination_txt_file}...")
    
    try:
        # 1. gzipライブラリを使って.fa.gzファイルを'rt'(テキスト読み込み)モードで開く
        with gzip.open(source_fa_file, 'rt') as f_read:
            # .read()でファイルの中身を全て読み込み、変数に保存する
            file_content = f_read.read()
        
        # 2. 保存先となる.txtファイルを'w'(書き込み)モードで開く
        with open(destination_txt_file, 'w') as f_write:
            # 読み込んだ内容をそのまま書き込む
            f_write.write(file_content)
        
        print(f"-> Success: Saved content of '{source_fa_file}' to '{destination_txt_file}'.")
    
    except FileNotFoundError:
        print(f"-> Error: Source file '{source_fa_file}' was not found. Skipping.")
    except Exception as e:
        print(f"-> An unexpected error occurred: {e}")

print("\nAll processing finished.")

In [ ]:
import os

# 1番から22番染色体、およびX, Y染色体を処理するリスト
# The list of chromosomes to process, from 1 to 22, plus X and Y.
chromosomes = list(range(1, 23)) + ['X', 'Y']
for n in chromosomes:
    source_file = f'references_GRCh37_chr{n}.txt'
    
    # 保存する新しいファイル名 (出力ディレクトリ内)
    destination_file = f'references_GRCh37_chr{n}_single_line.txt'

    print(f"Processing {source_file}...")

    try:
        sequence_lines = []
        # 通常のopenでテキストファイルを'r'(読み込み)モードで開く
        # Open the text file in read mode ('r') using the standard open function.
        with open(source_file, 'r') as f_read:
            # ファイルを1行ずつ読み込む
            # Read the file line by line.
            for line in f_read:
                # ヘッダー行('>'で始まる行)でなければ、リストに追加
                # If the line does not start with '>' (it's not a header), add it to the list.
                if not line.startswith('>'):
                    # 行末の改行文字などを削除して追加
                    # Append the line to the list after removing leading/trailing whitespace (including newlines).
                    sequence_lines.append(line.strip())
        
        # リストに格納した全ての塩基配列の行を、'' (空文字列)で連結して1行の文字列にする
        # Join all the sequence lines in the list into a single string with no separator.
        sequence_single_line = ''.join(sequence_lines)

        # 1行に繋げた配列を新しいファイルに書き出す
        # Write the resulting single-line string to the destination file.
        with open(destination_file, 'w') as f_write:
            f_write.write(sequence_single_line)

        print(f"-> Success: Saved sequence-only content to '{destination_file}'.")

    except FileNotFoundError:
        print(f"-> Error: Source file '{source_file}' was not found. Skipping.")
    except Exception as e:
        print(f"-> An unexpected error occurred: {e}")

print(f"\nAll processing finished. Check the '{output_dir}/' directory for results.")

## Deduplicate SNPs

In [ ]:
import pandas as pd
import os

# 処理したい染色体のリスト
chromosomes = list(range(1, 23))

# 元のファイルがあるディレクトリと、保存先のディレクトリ
source_directory = "../genome-data/gwas-id/"
output_directory = "../genome-data/gwas-id_deduplicated/"

# 保存先のディレクトリがなければ作成
os.makedirs(output_directory, exist_ok=True)

print("Starting deduplication process...")

for chrom in chromosomes:
    source_file = os.path.join(source_directory, f"gwas-id_chr{chrom}_genotypes.tsv")
    output_file = os.path.join(output_directory, f"gwas-id_chr{chrom}_genotypes_dedup.tsv")
    
    if os.path.exists(source_file):
        try:
            print(f"Processing {source_file}...")
            # SNPカタログを読み込む
            snp_df = pd.read_csv(source_file, sep='\t', low_memory=False)
            
            # POS列で重複している行を削除（最初の行を残す）
            snp_df_deduplicated = snp_df.drop_duplicates(subset=['POS'], keep='first')
            
            # 新しいファイルとして保存
            snp_df_deduplicated.to_csv(output_file, sep='\t', index=False)
            
            print(f"-> Saved deduplicated file to {output_file}")
            
        except Exception as e:
            print(f"-> An error occurred while processing {source_file}: {e}")
    else:
        print(f"-> Source file not found: {source_file}")

print("\nDeduplication process finished.")

## Make a list of protein coding genes

In [ ]:
gtf_path = "gencode.v19.annotation.gtf"
ensg2symbol = dict()

with open(gtf_path) as f:
    for line in f:
        if line.startswith("#"):
            continue
        fields = line.strip().split("\t")
        if len(fields) != 9:
            continue
        chrom, feature_type, start, end, score, strand, frame, attr = fields[0], fields[2], fields[3], fields[4], fields[5], fields[6], fields[7], fields[8]
        if feature_type != "gene":
            continue
        attr_dict = {}
        for a in attr.split(";"):
            a = a.strip()
            if a:
                k_v = a.split(" ", 1)
                if len(k_v) == 2:
                    key, value = k_v
                    attr_dict[key] = value.strip('"')
        ensg = attr_dict.get("gene_id")
        symbol = attr_dict.get("gene_name")
        if ensg and symbol:
            ensg2symbol[ensg.split('.')[0]] = symbol  # versionless

In [ ]:
txt_file = "./chrX_5000/genes/chrX_protein_coding_genes.txt"
gene_symbols = set()
with open(txt_file) as f:
    for l in f:
        gid = l.strip()
        if not gid:
            continue
        gid0 = gid.split('.')[0]  #v versionless
        if gid0 in ensg2symbol:
            gene_symbols.add(ensg2symbol[gid0])

print(f"{len(gene_symbols)} gene symbols mapped from {txt_file}")

In [ ]:
bed_file = "ucsc_expression.bed"
csv_file = "chrX_protein_coding_filtered.csv"

# BED の Gene Symbol 抽出
bed_symbols = set()
with open(bed_file) as bf:
    for line in bf:
        if line.strip() == "" or line.startswith("#"):
            continue
        parts = line.strip().split("\t")
        if len(parts) < 4:
            continue
        chr_field = parts[0]
        if chr_field not in ("chrX", "X"):
            continue
        bed_symbols.add(parts[3])

# CSV の Gene Symbol 抽出
csv_symbols = set()
import csv
with open(csv_file) as cf:
    reader = csv.DictReader(cf)
    for row in reader:
        csv_symbols.add(row["gene_symbol"])

# 比較
matched = csv_symbols & bed_symbols
unmatched = csv_symbols - bed_symbols
print(f"Matched: {len(matched)}")
print(f"Unmatched: {len(unmatched)}")
print("Unmatched genes:", unmatched)

# Analysis performed using AlphaGenome

In [ ]:
%%time
import numpy as np
import os
import random
import multiprocessing
import time
import csv
from alphagenome.data import genome
from alphagenome.models import dna_client
import pandas as pd  # 最小限の使用（CSV保存のみ）

# --- ワーカー関数 ---
def process_simulation_task(task_info):
    start_pos, simulation_num, chrom_num, full_ref_seq, snp_positions_array, lfc_dir, snp_dir, ontology = task_info

    max_retries = 3
    initial_backoff = 2.0

    for attempt in range(max_retries):
        try:
            model = dna_client.create(API_KEY)

            window_size = 1048576
            end_pos = min(start_pos + window_size, len(full_ref_seq))
            ref_sequence_slice = full_ref_seq[start_pos:end_pos]

            if ref_sequence_slice.startswith('N' * 100): return None
            if not set(ref_sequence_slice).issubset(set("ATGCN")): return None

            # SNP位置抽出（NumPyでフィルタ）
            mask = (snp_positions_array >= start_pos + 1) & (snp_positions_array <= end_pos)
            positions_in_window = snp_positions_array[mask]

            if len(positions_in_window) == 0: return None

            snp_count_to_generate = min(500, len(positions_in_window))
            selected_positions = random.sample(list(positions_in_window), snp_count_to_generate)

            alt_sequence_list = list(ref_sequence_slice[:1048575])  # ALTは1文字短く
            bases = "ACGT"
            snp_records = []

            for pos in selected_positions:
                index = pos - start_pos - 1
                if index >= len(alt_sequence_list): continue
                original_ref = alt_sequence_list[index]
                if original_ref == 'N': continue

                new_alt = random.choice(bases)
                alt_sequence_list[index] = new_alt
                snp_records.append(f"{pos}_{original_ref}_{new_alt}")

            alt_sequence_slice = "".join(alt_sequence_list)

            min_snp_pos, max_snp_pos = min(selected_positions), max(selected_positions)
            var_start_index = min_snp_pos - start_pos - 1
            var_end_index = max_snp_pos - start_pos
            var_ref_bases = ref_sequence_slice[var_start_index:var_end_index][:1048575]
            var_alt_bases = alt_sequence_slice[var_start_index:var_end_index][:1048575]

            interval = genome.Interval(chromosome=f"chr{chrom_num}", start=start_pos, end=end_pos)
            variant = genome.Variant(
                chromosome=f"chr{chrom_num}",
                position=min_snp_pos,
                reference_bases=var_ref_bases,
                alternate_bases=var_alt_bases
            )

            outputs = model.predict_variant(
                interval=interval, variant=variant, ontology_terms=[ontology],
                requested_outputs=[dna_client.OutputType.RNA_SEQ]
            )

            if outputs.reference.rna_seq.values.shape[1] > 0:
                ref_array = outputs.reference.rna_seq.values
                alt_array = outputs.alternate.rna_seq.values
                lfc_array = np.log2((alt_array + 1e-6) / (ref_array + 1e-6))

                lfc_npz_path = os.path.join(lfc_dir, f"output_chr{chrom_num}_{start_pos}_{simulation_num}.npz")
                np.savez_compressed(lfc_npz_path, lfc_data=lfc_array)

                snp_csv_path = os.path.join(snp_dir, f"snp_chr{chrom_num}_{start_pos}_{simulation_num}.csv")
                with open(snp_csv_path, 'w', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow(["SNP"])
                    for record in snp_records:
                        writer.writerow([record])

                return True

            return None

        except Exception as e:
            error_message = str(e)
            if ("RESOURCE_EXHAUSTED" in error_message or "UNAVAILABLE" in error_message) and attempt < max_retries - 1:
                wait_time = initial_backoff * (2 ** attempt) + random.uniform(0, 1)
                time.sleep(wait_time)
            else:
                return None

    return None

# --- メイン処理 ---
if __name__ == '__main__':
    API_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'
    window_size = 1048576
    num_simulations_per_window = 5000
    for CHROMOSOME_NUM in ["X"]:
        REF_FILE = f"../genome-data/GRCh37/references_GRCh37_chr{CHROMOSOME_NUM}_single_line.txt"
        SNP_CATALOG_FILE = f"../genome-data/gwas-id_deduplicated/gwas-id_chr{CHROMOSOME_NUM}_genotypes_dedup.tsv"
        ONTOLOGY = "UBERON:0013756"

        print("="*70)
        print(f"Starting analysis for Chromosome {CHROMOSOME_NUM}")
        print("="*70)

        LFC_NPZ_OUTPUT_DIR = f"./chr{CHROMOSOME_NUM}_5000/LFCdata"
        os.makedirs(LFC_NPZ_OUTPUT_DIR, exist_ok=True)
        SNP_CSV_OUTPUT_DIR = f"./chr{CHROMOSOME_NUM}_5000/SNPdata"
        os.makedirs(SNP_CSV_OUTPUT_DIR, exist_ok=True)

        try:
            with open(REF_FILE, 'r') as f:
                full_ref_seq = f.read().strip()
            chromosome_length = len(full_ref_seq)

            snp_catalog_df = pd.read_csv(SNP_CATALOG_FILE, sep='\t', usecols=["POS"], low_memory=False)
            snp_catalog_df['POS'] = pd.to_numeric(snp_catalog_df['POS'], errors='coerce')
            snp_catalog_df.dropna(subset=['POS'], inplace=True)
            snp_positions_array = snp_catalog_df['POS'].to_numpy(dtype=np.int64)
        except FileNotFoundError as e:
            print(f"Data file not found for chr{CHROMOSOME_NUM}, skipping. Error: {e}")
            continue

        window_start_positions = list(range(0, chromosome_length - window_size, window_size))
        tasks = [(
            start, sim_num, CHROMOSOME_NUM, full_ref_seq, snp_positions_array,
            LFC_NPZ_OUTPUT_DIR, SNP_CSV_OUTPUT_DIR, ONTOLOGY
        ) for start in window_start_positions for sim_num in range(1, num_simulations_per_window + 1)]

        num_processes = max(1, multiprocessing.cpu_count() - 8)
        print(f"Starting parallel processing for {len(tasks)} tasks with {num_processes} processes...")
        start_time = time.time()

        with multiprocessing.Pool(processes=num_processes) as pool:
            results = pool.map(process_simulation_task, tasks)

        end_time = time.time()
        success_count = sum(1 for r in results if r is not None)
        print(f"\nTotal parallel processing for chr{CHROMOSOME_NUM} finished in {end_time - start_time:.2f} seconds.")
        print(f"Successfully completed {success_count} out of {len(tasks)} tasks.")

    print("\n" + "="*70)
    print("All chromosome analyses are complete.")
    print("="*70)

In [ ]:
%%time
import numpy as np
import os
import random
import multiprocessing
import time
import csv
from alphagenome.data import genome
from alphagenome.models import dna_client
import pandas as pd  # 最小限の使用（CSV保存のみ）

# --- ワーカー関数 ---
def process_simulation_task(task_info):
    start_pos, simulation_num, chrom_num, full_ref_seq, snp_positions_array, lfc_dir, snp_dir, ontology = task_info

    max_retries = 3
    initial_backoff = 2.0

    for attempt in range(max_retries):
        try:
            model = dna_client.create(API_KEY)

            window_size = 1048576
            end_pos = min(start_pos + window_size, len(full_ref_seq))
            ref_sequence_slice = full_ref_seq[start_pos:end_pos]

            if not set(ref_sequence_slice).issubset(set("ATGCN")): return None

            # SNP位置抽出（NumPyでフィルタ）
            mask = (snp_positions_array >= start_pos + 1) & (snp_positions_array <= end_pos)
            positions_in_window = snp_positions_array[mask]

            if len(positions_in_window) == 0: return None

            snp_count_to_generate = min(500, len(positions_in_window))
            selected_positions = random.sample(list(positions_in_window), snp_count_to_generate)

            alt_sequence_list = list(ref_sequence_slice[:1048575])  # ALTは1文字短く
            bases = "ACGT"
            snp_records = []

            for pos in selected_positions:
                index = pos - start_pos - 1
                if index >= len(alt_sequence_list): continue
                original_ref = alt_sequence_list[index]
                if original_ref == 'N': continue

                new_alt = random.choice(bases)
                alt_sequence_list[index] = new_alt
                snp_records.append(f"{pos}_{original_ref}_{new_alt}")

            alt_sequence_slice = "".join(alt_sequence_list)

            min_snp_pos, max_snp_pos = min(selected_positions), max(selected_positions)
            var_start_index = min_snp_pos - start_pos - 1
            var_end_index = max_snp_pos - start_pos
            var_ref_bases = ref_sequence_slice[var_start_index:var_end_index][:1048575]
            var_alt_bases = alt_sequence_slice[var_start_index:var_end_index][:1048575]

            interval = genome.Interval(chromosome=f"chr{chrom_num}", start=start_pos, end=end_pos)
            variant = genome.Variant(
                chromosome=f"chr{chrom_num}",
                position=min_snp_pos,
                reference_bases=var_ref_bases,
                alternate_bases=var_alt_bases
            )

            outputs = model.predict_variant(
                interval=interval, variant=variant, ontology_terms=[ontology],
                requested_outputs=[dna_client.OutputType.RNA_SEQ]
            )

            if outputs.reference.rna_seq.values.shape[1] > 0:
                ref_array = outputs.reference.rna_seq.values
                alt_array = outputs.alternate.rna_seq.values
                lfc_array = np.log2((alt_array + 1e-6) / (ref_array + 1e-6))

                lfc_npz_path = os.path.join(lfc_dir, f"output_chr{chrom_num}_{start_pos}_{simulation_num}.npz")
                np.savez_compressed(lfc_npz_path, lfc_data=lfc_array)

                snp_csv_path = os.path.join(snp_dir, f"snp_chr{chrom_num}_{start_pos}_{simulation_num}.csv")
                with open(snp_csv_path, 'w', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow(["SNP"])
                    for record in snp_records:
                        writer.writerow([record])

                return True

            return None

        except Exception as e:
            error_message = str(e)
            if ("RESOURCE_EXHAUSTED" in error_message or "UNAVAILABLE" in error_message) and attempt < max_retries - 1:
                wait_time = initial_backoff * (2 ** attempt) + random.uniform(0, 1)
                time.sleep(wait_time)
            else:
                return None

    return None

# --- メイン処理 ---
if __name__ == '__main__':
    API_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'
    window_size = 1048576
    num_simulations_per_window = 5000
    for CHROMOSOME_NUM in ["X"]:
        REF_FILE = f"../genome-data/GRCh37/references_GRCh37_chr{CHROMOSOME_NUM}_single_line.txt"
        SNP_CATALOG_FILE = f"../genome-data/gwas-id_deduplicated/gwas-id_chr{CHROMOSOME_NUM}_genotypes_dedup.tsv"
        ONTOLOGY = "UBERON:0013756"

        print("="*70)
        print(f"Starting analysis for Chromosome {CHROMOSOME_NUM}")
        print("="*70)

        LFC_NPZ_OUTPUT_DIR = f"./chr{CHROMOSOME_NUM}_5000/LFCdata"
        os.makedirs(LFC_NPZ_OUTPUT_DIR, exist_ok=True)
        SNP_CSV_OUTPUT_DIR = f"./chr{CHROMOSOME_NUM}_5000/SNPdata"
        os.makedirs(SNP_CSV_OUTPUT_DIR, exist_ok=True)

        try:
            with open(REF_FILE, 'r') as f:
                full_ref_seq = f.read().strip()
            chromosome_length = len(full_ref_seq)

            snp_catalog_df = pd.read_csv(SNP_CATALOG_FILE, sep='\t', usecols=["POS"], low_memory=False)
            snp_catalog_df['POS'] = pd.to_numeric(snp_catalog_df['POS'], errors='coerce')
            snp_catalog_df.dropna(subset=['POS'], inplace=True)
            snp_positions_array = snp_catalog_df['POS'].to_numpy(dtype=np.int64)
        except FileNotFoundError as e:
            print(f"Data file not found for chr{CHROMOSOME_NUM}, skipping. Error: {e}")
            continue

        window_start_positions = list(range(0, min(3145728, chromosome_length - window_size), window_size))
        tasks = [(
            start, sim_num, CHROMOSOME_NUM, full_ref_seq, snp_positions_array,
            LFC_NPZ_OUTPUT_DIR, SNP_CSV_OUTPUT_DIR, ONTOLOGY
        ) for start in window_start_positions for sim_num in range(1, num_simulations_per_window + 1)]

        num_processes = max(1, multiprocessing.cpu_count() - 8)
        print(f"Starting parallel processing for {len(tasks)} tasks with {num_processes} processes...")
        start_time = time.time()

        with multiprocessing.Pool(processes=num_processes) as pool:
            results = pool.map(process_simulation_task, tasks)

        end_time = time.time()
        success_count = sum(1 for r in results if r is not None)
        print(f"\nTotal parallel processing for chr{CHROMOSOME_NUM} finished in {end_time - start_time:.2f} seconds.")
        print(f"Successfully completed {success_count} out of {len(tasks)} tasks.")

    print("\n" + "="*70)
    print("All chromosome analyses are complete.")
    print("="*70)

# Compute per-gene LFC averages

In [ ]:
#!/usr/bin/env python3
"""
extract_chrX_genes_singlewindow_region_preload_log.py

リージョン単位プリロード方式（親プロセスでそのリージョンの全 sim を読み込み、
Pool を作ってワーカーに処理させる。Linux の fork による COW を利用）

- CSV_FILE にある Gene Symbol のみ処理
- require_single モード：遺伝子がちょうど1つの window (= region) に完全に収まる遺伝子のみ処理
- 出力 per-gene npz: OUTPUT_DIR/<gene_symbol>.npz に 'LFC_sim' を (num_sims,1) で保存
- 複数ウィンドウにまたがる遺伝子は無視し、最後にリスト表示
- 保存されなかった遺伝子やデータなし遺伝子をリアルタイムで標準出力に表示
"""
import os, sys, glob, re, math, traceback
import numpy as np
from multiprocessing import Pool, cpu_count, get_start_method
from collections import Counter
import csv
import gc

# -------- CONFIG --------
LFC_NPZ_DIR = "chrX_5000/LFCdata"
BED_FILE = "ucsc_expression.bed"
CSV_FILE = "chrX_protein_coding_filtered.csv"
CHROMOSOME = "X"
WINDOW_SIZE = 1048576
OUTPUT_DIR = "chrX_5000/genes"
NPROCS = min(max(1, cpu_count()), 8)
# -----------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- read CSV Gene Symbol list ---
gene_symbols_set = set()
with open(CSV_FILE) as cf:
    reader = csv.DictReader(cf)
    for row in reader:
        gene_symbols_set.add(row["gene_symbol"])
print(f"Loaded {len(gene_symbols_set)} gene symbols from {CSV_FILE}")

# --- read BED and filter for chrX and genes in gene_symbols_set ---
bed_genes = []
with open(BED_FILE) as bf:
    for line in bf:
        if line.strip() == "" or line.startswith("#"):
            continue
        parts = line.strip().split("\t")
        if len(parts) < 4:
            continue
        chr_field, start_s, end_s, name = parts[0], parts[1], parts[2], parts[3]
        if chr_field not in (f"chr{CHROMOSOME}", CHROMOSOME):
            continue
        if name in gene_symbols_set:
            bed_genes.append((name, int(start_s), int(end_s)))
bed_genes.sort(key=lambda x: x[1])
print(f"Found {len(bed_genes)} genes in BED that match the gene list on chr{CHROMOSOME}")

if len(bed_genes) == 0:
    print("No genes found to process. Check BED and gene list matching.")
    sys.exit(1)

# --- parse LFC npz files ---
def tolerant_parse(fp):
    name = os.path.basename(fp).replace('.npz','')
    ints = [int(x) for x in re.findall(r'(\d+)', name)]
    if len(ints) >= 2:
        region = ints[0]
        sim = ints[-1] - 1
        return region, sim
    return None

lfc_files = sorted(glob.glob(os.path.join(LFC_NPZ_DIR, "*.npz")))
sim_files = {}
region_to_sims = {}
region_starts = set()
failed = 0
for fp in lfc_files:
    parsed = tolerant_parse(fp)
    if parsed is None:
        failed += 1
        continue
    region, sim = parsed
    if sim < 0:
        continue
    sim_files[(region, sim)] = fp
    region_starts.add(region)
    region_to_sims.setdefault(region, set()).add(sim)
region_starts = sorted(region_starts)
print(f"Parsed {len(sim_files)} LFC file entries, region_starts={len(region_starts)}, failed_parses={failed}")

if len(sim_files) == 0:
    print("No LFC files parsed. Check LFC_NPZ_DIR and filename pattern.")
    sys.exit(1)

# --- helper ---
GLOBAL_REGION_LFCS = None
GLOBAL_REGION_START = None
GLOBAL_NUM_SIMS = None
GLOBAL_WINDOW_SIZE = WINDOW_SIZE
GLOBAL_OUTPUT_DIR = OUTPUT_DIR

def init_worker(region_arrays, region_start, num_sims):
    global GLOBAL_REGION_LFCS, GLOBAL_REGION_START, GLOBAL_NUM_SIMS
    GLOBAL_REGION_LFCS = region_arrays
    GLOBAL_REGION_START = region_start
    GLOBAL_NUM_SIMS = num_sims

def process_gene_using_global(gene_tuple):
    try:
        gene_id, gstart, gend = gene_tuple
        out_path = os.path.join(GLOBAL_OUTPUT_DIR, f"{gene_id}.npz")
        if os.path.exists(out_path):
            return (gene_id, "skipped_exists", None)

        rs = GLOBAL_REGION_START
        re = rs + GLOBAL_WINDOW_SIZE
        if not (gstart >= rs and gend <= re):
            return (gene_id, "no_overlap_region", None)

        rel_start = gstart - rs
        rel_end = gend - rs
        out_arr = np.full((GLOBAL_NUM_SIMS, 1), np.nan, dtype=np.float32)
        any_found = False

        for sim_idx in range(GLOBAL_NUM_SIMS):
            arr = GLOBAL_REGION_LFCS[sim_idx]
            if arr is None or rel_end > len(arr):
                continue
            seg = arr[rel_start:rel_end]
            if seg.size == 0 or np.all(np.isnan(seg)):
                continue
            out_arr[sim_idx, 0] = float(np.nanmean(seg))
            any_found = True

        if not any_found:
            return (gene_id, "no_data_found", None)

        np.savez_compressed(out_path, LFC_sim=out_arr)
        return (gene_id, "ok", int(np.sum(np.isfinite(out_arr))))
    except Exception as e:
        tb = traceback.format_exc()
        return (gene_id, "worker_exception", str(tb))

# --- prepare genes by region ---
genes_by_region = {}
ignored_multiwindow_genes = []
for gene in bed_genes:
    name, s, e = gene
    matched = False
    for rs in region_starts:
        if s >= rs and e <= rs + WINDOW_SIZE:
            genes_by_region.setdefault(rs, []).append(gene)
            matched = True
            break
    if not matched:
        ignored_multiwindow_genes.append(name)

print(f"Genes fully contained in regions (per-region counts):")
tot_contained = 0
for rs in sorted(genes_by_region.keys()):
    cnt = len(genes_by_region[rs])
    tot_contained += cnt
    print(f"  region {rs}: {cnt} genes")
print(f"Total genes fully contained: {tot_contained}")
print(f"Total genes ignored upfront (not fully contained): {len(ignored_multiwindow_genes)}")

# --- multiprocessing start method ---
start_method = get_start_method(allow_none=True)
if start_method != 'fork':
    try:
        import multiprocessing as mp
        mp.set_start_method('fork', force=True)
        print("Set start method to 'fork' for better memory sharing.")
    except Exception:
        print("Warning: multiprocessing start method is not 'fork'.")

# --- region loop ---
all_results = []
skipped_count = 0

for region_start in region_starts:
    genes_in_region = genes_by_region.get(region_start, [])
    if not genes_in_region:
        continue
    sim_idx_set = sorted(region_to_sims.get(region_start, []))
    if not sim_idx_set:
        print(f"Region {region_start}: no sim files, skipping.")
        continue
    num_sims = max(sim_idx_set) + 1
    print(f"\n=== Processing region {region_start} (num_sims inferred = {num_sims}) ===")
    print(f"  Genes in region to process: {len(genes_in_region)}")

    # load sim arrays
    region_arrays = [None] * num_sims
    loaded = 0
    try:
        for sim in sim_idx_set:
            fp = sim_files.get((region_start, sim))
            if fp is None:
                continue
            with np.load(fp) as npz:
                arr = npz.get('lfc_data', None)
                if arr is None:
                    continue
                if arr.ndim == 2 and arr.shape[1] == 1:
                    arr = arr[:,0]
                region_arrays[sim] = np.ascontiguousarray(arr, dtype=np.float32)
                loaded += 1
        print(f"  Loaded {loaded}/{len(sim_idx_set)} sim arrays for region {region_start}.")
    except MemoryError:
        print("MemoryError while loading region arrays, skipping region.")
        del region_arrays
        gc.collect()
        continue

    print(f"  Creating pool with {NPROCS} workers for region {region_start} ...")
    pool = Pool(NPROCS, initializer=init_worker, initargs=(region_arrays, region_start, num_sims))
    try:
        results_region = pool.map(process_gene_using_global, genes_in_region)
    finally:
        pool.close()
        pool.join()

    # log results
    for gene_id, status, info in results_region:
        all_results.append((gene_id, status, info))
        if status == "skipped_exists":
            print(f"  Gene {gene_id}: already exists, skipped.")
        elif status == "no_data_found":
            print(f"  Gene {gene_id}: no LFC data found in region {region_start}, skipped.")
        elif status == "worker_exception":
            print(f"  Gene {gene_id}: worker exception:\n{info}")

    del region_arrays
    gc.collect()

# add ignored multiwindow genes to results
for g in ignored_multiwindow_genes:
    if not any(r[0] == g for r in all_results):
        all_results.append((g, "multiwindow_skipped", None))

# --- write summary CSV ---
summary_path = os.path.join(OUTPUT_DIR, "_summary.csv")
with open(summary_path, "w") as oh:
    oh.write("gene_id,status,info\n")
    for gid, status, info in all_results:
        oh.write(f"{gid},{status},{info}\n")

# --- summary ---
cnt = Counter([s for (_, s, _) in all_results])
print("\nDone. Summary:")
for k,v in cnt.items():
    print(f"  {k}: {v}")

if ignored_multiwindow_genes:
    print(f"\nIgnored genes spanning multiple windows (count={len(ignored_multiwindow_genes)}):")
    for g in ignored_multiwindow_genes:
        print(g)

print("Per-gene npz files written to:", OUTPUT_DIR)
print("Summary CSV:", summary_path)


# Plot

## Find SNP-sensitive genes and Plotting

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
violinplot_topN_with_imputation.py

- NPZ 内の LFC 配列に NaN が含まれていたらログ出力し、遺伝子内の平均で補完して処理します。
- 全シミュレーションで mean/std を計算 → 標準偏差上位 TOP_N を選択
- プロット時のみ NUM_SAMPLE 個をランダム抽出してviolinplot + points 表示
- 箱ひげ図を PNG（高解像度）で保存: violin_topN.png
"""
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from collections import defaultdict

# ---------- CONFIG ----------
BED_FILE = "ucsc_expression.bed"
SUMMARY_FILE = "chrX_5000/genes/_summary_updated.csv"
NPZ_DIR = "chrX_5000/genes"
TOP_N = 10            # 標準偏差上位何個を表示
NUM_SAMPLE = 100      # プロット用ダウンサンプリング数
SEED = 1
IMPUTE_METHOD = "mean"  # "mean" または "median"
# ----------------------------

random.seed(SEED)
np.random.seed(SEED)
sns.set(style="whitegrid")

# --- BED読み込み（順序保持） ---
bed_genes = []
with open(BED_FILE) as f:
    for line in f:
        if line.startswith("#") or line.strip() == "":
            continue
        parts = line.strip().split("\t")
        if len(parts) < 4:
            continue
        chrom, start, end, gene = parts[0], int(parts[1]), int(parts[2]), parts[3]
        bed_genes.append((gene, start, end))
bed_genes.sort(key=lambda x: x[1])
gene_order = [g[0] for g in bed_genes]

# --- summary 読み込み & ok または skipped_exists の遺伝子のみ ---
ok_genes = set()
with open(SUMMARY_FILE) as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row.get("status") in ("ok", "skipped_exists"):
            ok_genes.add(row.get("gene_id"))

# BEDの順序で候補遺伝子リスト
plot_genes = [g for g in gene_order if g in ok_genes]
print(f"Candidate genes from BED+SUMMARY: {len(plot_genes)}")

# --- npz 読み込み（形状チェック・NaN検出→補完） ---
lfc_dict = {}
nan_info = []
missing_files = []
shape_counts = defaultdict(int)

for gene in plot_genes:
    npz_path = os.path.join(NPZ_DIR, f"{gene}.npz")
    if not os.path.exists(npz_path):
        missing_files.append(gene)
        continue
    try:
        with np.load(npz_path) as npz:
            key = "LFC_sim" if "LFC_sim" in npz else (list(npz.keys())[0] if len(npz.keys()) > 0 else None)
            if key is None:
                missing_files.append(gene)
                continue
            arr = np.asarray(npz[key]).squeeze()
            if arr.ndim != 1:
                print(f"Warning: {gene}.npz -> array ndim={arr.ndim}, skipping")
                missing_files.append(gene)
                continue
            n_nan = int(np.isnan(arr).sum())
            if n_nan > 0:
                if n_nan == arr.size:
                    print(f"[SKIP] {gene}: all {n_nan} values are NaN -> skipped")
                    missing_files.append(gene)
                    continue
                fill_val = float(np.nanmedian(arr)) if IMPUTE_METHOD == "median" else float(np.nanmean(arr))
                print(f"[IMPUTE] {gene}: {n_nan} NaN(s) found. Filling with {IMPUTE_METHOD}={fill_val:.6g}")
                arr = np.where(np.isnan(arr), fill_val, arr)
                nan_info.append((gene, n_nan, arr.size))
            lfc_dict[gene] = arr
            shape_counts[len(arr)] += 1
    except Exception as e:
        print(f"[ERROR] Failed to load {npz_path}: {e}")
        missing_files.append(gene)

print(f"Loaded {len(lfc_dict)} genes with usable LFC arrays.")
if missing_files:
    print(f"Missing/unusable files for {len(missing_files)} genes (examples): {missing_files[:10]}")
if nan_info:
    print(f"Imputed NaN for {len(nan_info)} genes (examples): {nan_info[:10]}")

# --- simulation length check & truncate ---
unique_lengths = sorted(shape_counts.keys())
if len(unique_lengths) == 0:
    raise RuntimeError("No valid LFC arrays loaded.")
if len(unique_lengths) > 1:
    print(f"Warning: multiple simulation lengths detected: {unique_lengths}")
    min_len = min(unique_lengths)
    print(f"Truncating to {min_len}.")
    for g, arr in list(lfc_dict.items()):
        if len(arr) > min_len:
            lfc_dict[g] = arr[:min_len]
else:
    min_len = unique_lengths[0]
    print(f"All arrays length = {min_len}")

# --- mean/std 計算 ---
means = {g: float(np.mean(v)) for g, v in lfc_dict.items()}
stds  = {g: float(np.std(v))  for g, v in lfc_dict.items()}

# --- 標準偏差降順で TOP_N 選択 ---
top_genes = sorted(lfc_dict.keys(), key=lambda g: stds[g], reverse=True)[:TOP_N]
print(f"Top {TOP_N} genes by std: {top_genes}")

# --- ダウンサンプリング（プロットのみ） ---
num_total_sims = min_len
sample_indices = sorted(random.sample(range(num_total_sims), min(NUM_SAMPLE, num_total_sims)))
plot_rows = [{"遺伝子名": g, "LFC": float(lfc_dict[g][i])}
             for g in top_genes for i in sample_indices]
df_plot = pd.DataFrame(plot_rows)

# --- 箱ひげ図に変更 ---
plt.figure(figsize=(max(8, TOP_N*1.8), 6))
sns.set_context('poster')
ax = sns.violinplot(x="遺伝子名", y="LFC", data=df_plot)
sns.stripplot(x="遺伝子名", y="LFC", data=df_plot, color="black", size=3, jitter=True, ax=ax)

plt.xticks(rotation=45, ha='right')
plt.xlabel("Genes")
plt.ylabel("LFC")
plt.title(f"Downsampled LFC for top {TOP_N} genes by std dev")
plt.tight_layout()

# 🔽 高解像度PNGで保存
plt.savefig("violin_topN.png", dpi=300, bbox_inches="tight")

plt.show()

# --- 統計テーブル ---
df_stats = pd.DataFrame({
    "遺伝子名": top_genes,
    "平均値": [means[g] for g in top_genes],
    "標準偏差": [stds[g] for g in top_genes],
})
print("\n=== Top genes statistics (computed on all simulations, with imputation) ===")
display(df_stats)

## Calculate SD and RMSE and Making Figs

In [ ]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# ====== 設定 ======
data_dir = "chrX_5000/genes/"
n_list = [2, 3, 4, 5, 6, 7, 8, 9, 10,
          20, 30, 40, 50, 60, 70, 80, 90, 100,
          200, 300, 400, 500, 600, 700, 800, 900,
          1000, 2000, 3000, 4000, 5000]
seed = 123
rng = np.random.default_rng(seed)
output_png = "sd_rmse_mean95ci_noscatter.png"

sns.set(style='ticks', context='poster')  # 👈 見やすく調整

# ====== データ読み込み (.npz only) ======
files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir)
                if f.lower().endswith(".npz")])
if len(files) == 0:
    raise FileNotFoundError("No .npz files found in " + data_dir)

gene_names = [os.path.basename(f) for f in files]

gene_list = []
for f in files:
    with np.load(f) as d:
        key = list(d.keys())[0]
        arr = np.asarray(d[key]).ravel()
        gene_list.append(arr)
gene_matrix = np.stack(gene_list, axis=0)
G, M = gene_matrix.shape
print(f"Loaded {G} genes, simulations per gene = {M}")

true_means = np.nanmean(gene_matrix, axis=1)

nan_genes = [gene_names[i] for i in range(G) if np.isnan(gene_matrix[i]).any()]
if nan_genes:
    print(f"Number of genes with NaN: {len(nan_genes)}")

sd_scatter_x, sd_scatter_y = [], []
rmse_scatter_x, rmse_scatter_y = [], []
valid_ns = []

for n in n_list:
    if n > M:
        continue

    indices = rng.choice(M, size=n, replace=True)
    sel = gene_matrix[:, indices]

    per_gene_sd = np.nanstd(sel, axis=1, ddof=1)
    per_gene_mean = np.nanmean(sel, axis=1)
    per_gene_rmse = np.sqrt((per_gene_mean - true_means) ** 2)

    sd_scatter_x.extend([n] * G)
    sd_scatter_y.extend(per_gene_sd.tolist())
    rmse_scatter_x.extend([n] * G)
    rmse_scatter_y.extend(per_gene_rmse.tolist())

    valid_ns.append(n)
    #print(f"Processed n={n}")

df_sd = pd.DataFrame({'n': sd_scatter_x, 'SD': sd_scatter_y}).dropna()
df_rmse = pd.DataFrame({'n': rmse_scatter_x, 'RMSE': rmse_scatter_y}).dropna()

# ====== プロット ======
plt.figure(figsize=(12, 10))

# --- 上: SD ---
ax1 = plt.subplot(2,1,1)
sns.lineplot(data=df_sd, x='n', y='SD',
             ax=ax1,
             ci=95,
             #err_style='band',
             marker='o',
             markersize=8,
             color='tab:blue',
             #err_kws={'ecolor': 'black', 'capsize': 5},
             markeredgecolor='k',
             label='mean ± 95%CI')
ax1.set_xscale('log')
ax1.set_xlabel('Number of selected simulations (n, log scale)') 
ax1.set_ylabel('SD of LFC')
ax1.set_ylim(0.0, 0.1)
ax1.set_title('Per-gene SD (mean ± 95% CI)')
ax1.grid(True, which='both', ls='--', alpha=0.3)
ax1.legend()

# --- 下: RMSE ---
ax2 = plt.subplot(2,1,2, sharex=ax1)
sns.lineplot(data=df_rmse, x='n', y='RMSE',
             ax=ax2,
             ci=95,
             #err_style='band',
             marker='o',
             markersize=8,
             color='tab:orange',
             #err_kws={'ecolor': 'black', 'capsize': 5},
             markeredgecolor='k',
             label='mean ± 95%CI')
ax2.set_xscale('log')
ax2.set_xlabel('Number of selected simulations (n, log scale)')
ax2.set_ylabel('LFC RMSE\n(vs full-sim mean)')
ax2.set_ylim(0.0, 0.05)
ax2.set_title('Per-gene RMSE (vs full-sim mean)')
ax2.grid(True, which='both', ls='--', alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.savefig(output_png, dpi=300)
print(f"Saved figure to {output_png}")
plt.show()

## Find top SNPs that influence gene expression

### Linear regression is used to narrow down the SNPs, and Random Forest is used to find SNP candidates that contribute to gene expression changes.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
multiwindow_snp_lfc_pipeline_numpy.py

- pandasを使わずnumpyのみで処理・保存
- per-window CSVと全ウィンドウ集約CSVを保存
"""
import os
import glob
import time
import numpy as np
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy import sparse

# ---------------- CONFIG ----------------
BED_FILE = "ucsc_expression.bed"
NPZ_DIR = "chrX_5000/genes"
SNP_DIR = "chrX_5000/SNPdata"
WINDOW_SIZE = 1048576
TOP_LINEAR_SNP = 500
SEED = 42
USE_SPARSE_THRESHOLD = 30000
MAX_LINEAR_DENSE = 50000
OUTPUT_DIR = "windows_topSNPs_rf"
SAVE_PLOTS = False
# -----------------------------------------

np.random.seed(SEED)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- load BED and restrict to chrX ---
bed_genes = []
with open(BED_FILE) as bf:
    for line in bf:
        if line.startswith("#") or line.strip() == "":
            continue
        parts = line.strip().split("\t")
        if len(parts) < 4:
            continue
        chrom, start_s, end_s, gene = parts[0], parts[1], parts[2], parts[3]
        if chrom != "chrX":
            continue
        try:
            start_i = int(start_s)
            end_i = int(end_s)
        except:
            continue
        bed_genes.append((start_i, end_i, gene))
if len(bed_genes) == 0:
    raise RuntimeError("No chrX genes found in BED_FILE")

# compute window starts
min_start = min(s for s, e, g in bed_genes)
max_end = max(e for s, e, g in bed_genes)
window_starts = list(range((min_start // WINDOW_SIZE) * WINDOW_SIZE, max_end + WINDOW_SIZE, WINDOW_SIZE))
print(f"chrX windows to process: {len(window_starts)} windows from {window_starts[0]} to {window_starts[-1]}")

def genes_in_window(wstart):
    wend = wstart + WINDOW_SIZE
    return [g for s,e,g in bed_genes if s >= wstart and s < wend]

# aggregate results
all_results_rows = []
all_stats = []

start_time = time.time()

for wstart in window_starts:
    wend = wstart + WINDOW_SIZE
    genes = genes_in_window(wstart)
    if len(genes) == 0:
        continue
    print("\n" + "="*60)
    print(f"Window {wstart}-{wend}: {len(genes)} genes")

    # --- load gene LFC arrays ---
    lfc_list = []
    valid_genes = []
    for gene in genes:
        npz_path = os.path.join(NPZ_DIR, f"{gene}.npz")
        if not os.path.exists(npz_path):
            continue
        try:
            with np.load(npz_path) as npz:
                key = list(npz.keys())[0]
                arr = np.asarray(npz[key]).ravel()
        except:
            continue
        if arr.size == 0:
            continue
        if np.isnan(arr).any():
            arr = np.where(np.isnan(arr), np.nanmean(arr), arr)
        lfc_list.append(arr)
        valid_genes.append(gene)
    if len(lfc_list) == 0:
        print(f"[WARN] no usable gene LFC arrays in window {wstart}")
        continue
    lfc_matrix = np.stack(lfc_list, axis=0)
    G, M = lfc_matrix.shape
    print(f"Loaded LFC for {G} genes, simulations per gene = {M}")

    # y = sum(|LFC|)
    y = np.sum(np.abs(lfc_matrix), axis=0)
    print(f"y mean={np.nanmean(y):.4g}, max={np.nanmax(y):.4g}")

    # --- load SNP files ---
    snp_pattern = os.path.join(SNP_DIR, f"snp_chrX_{wstart}_*.csv")
    snp_files = sorted(glob.glob(snp_pattern))
    if len(snp_files) == 0:
        print(f"[INFO] no SNP files for window {wstart}")
        continue
    print(f"Found {len(snp_files)} SNP files")

    all_snps = []
    sim_count = 0
    for sf in snp_files:
        try:
            snps = np.loadtxt(sf, dtype=str, delimiter=",")
            if snps.ndim == 0:
                snps = np.array([snps])
        except:
            snps = np.array([])
        all_snps.extend(snps.tolist())
        sim_count += 1

    unique_snps = sorted(set(all_snps))
    num_unique = len(unique_snps)
    total_occurrences = len(all_snps)
    print(f"Unique SNPs: {num_unique}, total occurrences: {total_occurrences}")
    if num_unique == 0:
        continue

    # build X
    snp_to_idx = {s:i for i,s in enumerate(unique_snps)}
    use_sparse = (num_unique > USE_SPARSE_THRESHOLD)
    if not use_sparse:
        X = np.zeros((sim_count, num_unique), dtype=np.int8)
        for sim_idx, sf in enumerate(snp_files):
            try:
                snps = np.loadtxt(sf, dtype=str, delimiter=",")
                if snps.ndim == 0:
                    snps = np.array([snps])
                for s in snps:
                    X[sim_idx, snp_to_idx[s]] = 1
            except:
                continue
    else:
        rows, cols, data = [], [], []
        for sim_idx, sf in enumerate(snp_files):
            try:
                snps = np.loadtxt(sf, dtype=str, delimiter=",")
                if snps.ndim == 0:
                    snps = np.array([snps])
                for s in snps:
                    idx = snp_to_idx.get(s)
                    if idx is not None:
                        rows.append(sim_idx)
                        cols.append(idx)
                        data.append(1)
            except:
                continue
        if len(rows) == 0:
            continue
        X = sparse.coo_matrix((data, (rows, cols)), shape=(sim_count, num_unique), dtype=np.int8).tocsr()
    print(f"X shape: {X.shape}")

    # --- linear regression screening ---
    coef = None
    if (not use_sparse) and (num_unique <= MAX_LINEAR_DENSE):
        try:
            model_lin = LinearRegression()
            model_lin.fit(X, y)
            coef = model_lin.coef_
        except:
            pass
    if coef is None:
        sgd = SGDRegressor(penalty='l2', max_iter=2000, tol=1e-4, random_state=SEED)
        sgd.fit(X, y)
        coef = sgd.coef_

    abscoef = np.abs(coef)
    sorted_idx = np.argsort(abscoef)[::-1]
    top_k = min(TOP_LINEAR_SNP, num_unique)
    top_idx = sorted_idx[:top_k]
    top_snps = [unique_snps[i] for i in top_idx]

    # --- RandomForest on top SNPs ---
    if sparse.issparse(X):
        X_top = X[:, top_idx].toarray()
    else:
        X_top = X[:, top_idx]
    rf = RandomForestRegressor(n_estimators=200, random_state=SEED, n_jobs=-1)
    rf.fit(X_top, y)
    importances = rf.feature_importances_

    # --- collect results ---
    res = sorted(zip(top_snps, importances, coef[top_idx]), key=lambda x: x[1], reverse=True)

    # --- numpy arrayでper-window保存 ---
    res_array = np.array([[r[0], r[1], r[2]] for r in res], dtype=object)
    out_csv = os.path.join(OUTPUT_DIR, f"window_{wstart}_topSNPs_rf.csv")
    np.savetxt(out_csv, res_array, fmt="%s", delimiter=",",
               header="SNP,RF_importance,linear_coef", comments='')
    print(f"Saved per-window results: {out_csv}")

    # --- append to global results ---
    for snp, imp, lincoef in res:
        pos = int(snp.split("_")[0]) if "_" in snp else None
        all_results_rows.append([wstart, snp, pos, imp, lincoef, num_unique, total_occurrences, G])

    all_stats.append([wstart, G, num_unique, total_occurrences, float(np.nanmean(y)), float(np.nanmax(y))])

# --- save aggregated results ---
if len(all_results_rows) > 0:
    all_results_array = np.array(all_results_rows, dtype=object)
    out_all_csv = os.path.join(OUTPUT_DIR, "all_windows_topSNPs_rf.csv")
    np.savetxt(out_all_csv, all_results_array, fmt="%s", delimiter=",",
               header="window_start,SNP,pos,RF_importance,linear_coef,num_unique_snps,total_snp_occurrences,num_genes_in_window",
               comments='')
    print(f"\nSaved aggregated results to {out_all_csv}")

# --- save stats ---
if len(all_stats) > 0:
    all_stats_array = np.array(all_stats, dtype=object)
    out_stats_csv = os.path.join(OUTPUT_DIR, "windows_stats_summary.csv")
    np.savetxt(out_stats_csv, all_stats_array, fmt="%s", delimiter=",",
               header="window_start,num_genes,num_unique_snps,total_snp_occurrences,y_mean,y_max", comments='')
    print(f"Saved window stats to {out_stats_csv}")

elapsed = time.time() - start_time
print(f"\nFinished processing. elapsed {elapsed:.1f} sec")


### Make a Fig

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
plot_windows_rf.py

- Read per-window CSV files produced by the pipeline in OUTPUT_DIR = "windows_topSNPs_rf"
- For each window file window_{wstart}_topSNPs_rf.csv:
    - plot SNP position vs RF_importance and save window_{wstart}_rf_scatter.png
- After per-window plotting, aggregate all windows and save an all-windows scatter:
    - all_windows_rf_scatter.png
"""
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

OUTPUT_DIR = "windows_topSNPs_rf"
PER_WINDOW_PATTERN = os.path.join(OUTPUT_DIR, "window_*_topSNPs_rf.csv")
PER_WINDOW_PNG_TEMPLATE = os.path.join(OUTPUT_DIR, "window_{wstart}_rf_scatter.png")
ALL_PNG = os.path.join(OUTPUT_DIR, "all_windows_rf_scatter.png")

sns.set(style='ticks', context='poster')

# find CSV files
csv_files = sorted(glob.glob(PER_WINDOW_PATTERN))
if len(csv_files) == 0:
    raise FileNotFoundError(f"No per-window CSVs found with pattern {PER_WINDOW_PATTERN}")

all_positions = []
all_importances = []
all_windows = []

print(f"Found {len(csv_files)} per-window CSV files. Processing...")

for csv_path in csv_files:
    try:
        # try reading with numpy genfromtxt (skip header)
        data = np.genfromtxt(csv_path, delimiter=",", dtype=str, skip_header=1)
    except Exception as e:
        print(f"[WARN] failed to read {csv_path}: {e}")
        continue

    # handle cases: single-line file (returns 1d array) or empty
    if data.size == 0:
        print(f"[INFO] empty or no-data file, skipping: {csv_path}")
        continue

    # Ensure 2D
    if data.ndim == 1:
        data = data.reshape(1, -1)

    # Expect columns: SNP,RF_importance,linear_coef
    # SNP is data[:,0], RF_importance is data[:,1]
    snp_col = data[:, 0].astype(str)
    try:
        rf_col = data[:, 1].astype(float)
    except Exception:
        # try to clean commas or quotes
        rf_col = np.array([float(x.strip().strip('"').strip("'")) if x not in [None, "nan", ""] else np.nan for x in data[:,1]])

    # extract positions (take numeric prefix before first underscore)
    positions = []
    for s in snp_col:
        try:
            pos = int(s.split("_")[0])
        except Exception:
            # if cannot parse, skip this SNP entry
            pos = None
        positions.append(pos)

    # filter invalid
    pos_arr = np.array([p for p in positions if p is not None], dtype=int)
    imp_arr = np.array([rf_col[i] for i,p in enumerate(positions) if p is not None], dtype=float)

    if pos_arr.size == 0:
        print(f"[INFO] no valid SNP positions in {csv_path}, skipping plotting for this file.")
        continue

    # per-window plotting
    # determine window start from filename: window_{wstart}_topSNPs_rf.csv
    fname = os.path.basename(csv_path)
    try:
        # parse numeric chunk after "window_" and before "_top"
        prefix = fname.split("_topSNPs_rf")[0]
        wstart_str = prefix.replace("window_", "")
        wstart = int(wstart_str)
    except Exception:
        wstart = None

    # 1) per-window scatter plot
    plt.figure(figsize=(10,3))
    plt.scatter(pos_arr, imp_arr, s=18, alpha=0.7, edgecolors='none')
    plt.xlabel("Genomic position (bp)")
    plt.ylabel("RF importance")
    title = f"Window {wstart}" if wstart is not None else fname
    plt.title(title)
    if wstart is not None:
        plt.xlim(wstart, wstart + 1048576)  # window size hard-coded as in pipeline
    plt.grid(True, linestyle='--', alpha=0.4)
    png_path = PER_WINDOW_PNG_TEMPLATE.format(wstart=wstart if wstart is not None else fname.replace(".csv",""))
    plt.tight_layout()
    plt.savefig(png_path, dpi=200)
    plt.close()
    print(f"Saved per-window plot: {png_path}")

    # append to aggregate lists (for summary plot)
    all_positions.extend(pos_arr.tolist())
    all_importances.extend(imp_arr.tolist())
    all_windows.extend([wstart] * len(pos_arr))

# --- summary plot across all windows ---
if len(all_positions) == 0:
    print("No SNP data collected from per-window CSVs; skipping aggregate plot.")
else:
    all_positions = np.array(all_positions, dtype=int)
    all_importances = np.array(all_importances, dtype=float)
    all_windows = np.array(all_windows, dtype=object)

    # Sort by position for nicer plotting (optional)
    order = np.argsort(all_positions)
    all_positions = all_positions[order]
    all_importances = all_importances[order]
    all_windows = all_windows[order]

    plt.figure(figsize=(18,4))
    plt.scatter(all_positions, all_importances, s=12, alpha=0.5, edgecolors='none')
    plt.xlabel("Genomic position (bp)")
    plt.ylabel("RF importance")
    plt.title("All windows: SNP RF importance across chrX (preliminary)")
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.savefig(ALL_PNG, dpi=200)
    plt.show()
    print(f"Saved aggregate plot: {ALL_PNG}")

## Make a Fig(Privacy Risk - Number of Simulation)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='ticks', context='poster')

# パラメータ
n = 2200       # ウィンドウ内SNP数
x = 375        # 1回のインプットで変更するSNP数

# リスク閾値pの範囲（対数空間）
p_vals = np.logspace(-50, -1, 200)
y_vals = np.log(p_vals) / np.log(1 - (x/n))

# --- 交点を求める関数 ---
def find_p_for_y(target_y, p_vals, y_vals):
    idx = np.argmin(np.abs(y_vals - target_y))
    return p_vals[idx], y_vals[idx]

# 交点を求める
p25, y25 = find_p_for_y(25, p_vals, y_vals)
p250, y250 = find_p_for_y(250, p_vals, y_vals)

# --- プロット ---
plt.figure(figsize=(10.5,6))
plt.plot(p_vals, y_vals, color='blue', lw=2, label="y = log(p) / log(1 - x/n)")

# y=25, y=250 の水平線
plt.axhline(25, color='orange', linestyle='--', label='y = 25')
plt.axhline(250, color='green', linestyle='--', label='y = 250')

# 交点（縦線＋点マーカー）
plt.axvline(p25, color='orange', linestyle=':')
plt.axvline(p250, color='green', linestyle=':')
plt.scatter([p25, p250], [y25, y250], color='red', zorder=5)

# 交点ラベル（パーセント表記）
plt.text(p25, y25 + 5, f"p ≈ {p25*100:.3g}%", color='black', ha='left')
plt.text(p250, y250 + 5, f"p ≈ {p250*100:.3g}%", color='black', ha='left')

# 軸とタイトル
plt.xscale('log')
plt.xlabel("Risk threshold p (log scale)")
plt.ylabel("Required number of \n input sequences y")
plt.title("Required y vs risk threshold p (x = 375, n = 2200)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.savefig("risk assessment.png", dpi=300)
plt.show()